# Orbit Computation

This tutorial demonstrates how to generate satellite orbits using various models.

## Setup

In [2]:
import numpy as np
import pandas as pd

import plotly.graph_objs as go

from ostk.physics.units import Length
from ostk.physics.units import Angle
from ostk.physics.time import Scale
from ostk.physics.time import Instant
from ostk.physics.time import Duration
from ostk.physics.time import Interval
from ostk.physics.time import DateTime
from ostk.physics.coordinate.spherical import LLA
from ostk.physics.coordinate import Frame
from ostk.physics import Environment
from ostk.physics.environment.objects.celestial_bodies import Earth

from ostk.astrodynamics import Trajectory
from ostk.astrodynamics.trajectory import Orbit
from ostk.astrodynamics.trajectory.orbit.models import Kepler
from ostk.astrodynamics.trajectory.orbit.models.kepler import COE
from ostk.astrodynamics.trajectory.orbit.models import SGP4
from ostk.astrodynamics.trajectory.orbit.models.sgp4 import TLE

---

## SGP4

### Computation

In [ ]:
environment = Environment.default()

Create a Classical Orbital Element (COE) set:

In [ ]:
a = Length.kilometers(7000.0)
e = 0.0001
i = Angle.degrees(35.0)
raan = Angle.degrees(40.0)
aop = Angle.degrees(45.0)
nu = Angle.degrees(50.0)

coe = COE(a, e, i, raan, aop, nu)

Setup a Keplerian orbital model:

In [ ]:
epoch = Instant.date_time(DateTime(2022, 1, 1, 0, 0, 0), Scale.UTC)
earth = environment.access_celestial_object_with_name('Earth')

keplerian_model = Kepler(coe, epoch, earth, Kepler.PerturbationType.No)

Create a Two-Line Element (TLE) set:

In [ ]:
tle = TLE(
    '1 48915U 21059AN  21347.87548576  .00000537  00000-0  34080-4 0  9995',
    '2 48915  97.5301 114.2938 0012149  20.2088 339.9621 15.12988721 25867'
)

Setup a SGP4 orbital model:

In [ ]:
sgp4_model = SGP4(tle)

Setup the orbit:

In [ ]:
# orbit = Orbit(keplerian_model, earth)
orbit = Orbit(sgp4_model, earth)

Now that the orbit is set, we can compute the satellite position:

In [ ]:
start_instant = Instant.date_time(DateTime(2021, 12, 14, 0, 0, 0), Scale.UTC)
end_instant = Instant.date_time(DateTime(2021, 12, 15, 0, 0, 0), Scale.UTC)

In [ ]:
interval = Interval.closed(start_instant, end_instant)

In [ ]:
step = Duration.minutes(1.0)

Generate a time grid:

In [ ]:
instants = interval.generate_grid(step)

In [ ]:
states = [[instant, orbit.get_state_at(instant)] for instant in instants]

In [ ]:
def convert_state (instant, state):
    
    lla = LLA.cartesian(
        state.get_position().in_frame(Frame.ITRF(), state.get_instant()).get_coordinates(), 
        Earth.equatorial_radius, 
        Earth.flattening
    )
    
    return [
                repr(instant),
                float(instant.get_modified_julian_date(Scale.UTC)),
                *state.get_position().get_coordinates().tolist(),
                *state.get_velocity().get_coordinates().tolist(),
                float(lla.get_latitude().in_degrees()),
                float(lla.get_longitude().in_degrees()),
                float(lla.get_altitude().in_meters())
            ]

In [ ]:
orbit_data = [convert_state(instant, state) for [instant, state] in states]

In [ ]:
orbit_df = pd.DataFrame(
    orbit_data, 
    columns = [
        '$Time^{UTC}$', 
        '$MJD^{UTC}$', 
        '$x_{x}^{ECI}$', 
        '$x_{y}^{ECI}$', 
        '$x_{z}^{ECI}$', 
        '$v_{x}^{ECI}$', 
        '$v_{y}^{ECI}$', 
        '$v_{z}^{ECI}$', 
        '$Latitude$', 
        '$Longitude$', 
        '$Altitude$'
    ]
)

### Output

Table:

In [ ]:
orbit_df.head()

2D plot, over **World Map**:

In [ ]:
figure = go.Figure(
    data = go.Scattergeo(
        lon = orbit_df['$Longitude$'],
        lat = orbit_df['$Latitude$'],
        mode = 'lines',
        line = go.scattergeo.Line(
            width = 1,
            color = 'red'
        )
    ),
    layout = go.Layout(
        title = None,
        showlegend = False,
        height=1000,
        geo = go.layout.Geo(
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)'
        )
    )
)

figure.show()

3D plot, in **Earth Fixed** frame:

In [ ]:
figure = go.Figure(
    data = [
        go.Scattergeo(
            lon = orbit_df['$Longitude$'],
            lat = orbit_df['$Latitude$'],
            mode = 'lines',
            line = go.scattergeo.Line(
                width = 2,
                color = 'rgb(255, 62, 79)'
            )
        )
    ],
    layout = go.Layout(
        title = None,
        showlegend = False,
        width = 800,
        height = 800,
        geo = go.layout.Geo(
            showland = True,
            showlakes = True,
            showcountries = False,
            showocean = True,
            countrywidth = 0.0,
            landcolor = 'rgb(100, 100, 100)',
            lakecolor = 'rgb(240, 240, 240)',
            oceancolor = 'rgb(240, 240, 240)',
            projection = dict( 
                type = 'orthographic',
                rotation = dict(
                    lon = -100,
                    lat = 40,
                    roll = 0
                )            
            ),
            lonaxis = dict( 
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)',
                gridwidth = 0.5
            ),
            lataxis = dict( 
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)',
                gridwidth = 0.5
            )
        )
    )
)

figure.show()

3D plot, in **Earth Inertial** frame:

In [ ]:
theta = np.linspace(0, 2 * np.pi, 30)
phi = np.linspace(0, np.pi, 30)

theta_grid, phi_grid = np.meshgrid(theta, phi)

r = float(Earth.equatorial_radius.in_meters())

x = r * np.cos(theta_grid) * np.sin(phi_grid)
y = r * np.sin(theta_grid) * np.sin(phi_grid)
z = r * np.cos(phi_grid)

earth = go.Surface(
    x=x,
    y=y,
    z=z,
    colorscale='Viridis',
    showscale=False
)

trace = go.Scatter3d(
    x=orbit_df['$x_{x}^{ECI}$'],
    y=orbit_df['$x_{y}^{ECI}$'],
    z=orbit_df['$x_{z}^{ECI}$'],
    mode='lines',
    marker=dict(
        size=0,
        color=orbit_df['$x_{z}^{ECI}$'],
        colorscale='Viridis',
        showscale=False
    ),
    line=dict(
        color=orbit_df['$x_{z}^{ECI}$'],
        width=1
    )
)

figure = go.Figure(
    data = [earth, trace],
    layout = go.Layout(
        title = None,
        width = 800,
        height = 1000,
        showlegend = False,
        scene = go.layout.Scene(
            xaxis = dict(
                gridcolor = 'rgb(255, 255, 255)',
                zerolinecolor = 'rgb(255, 255, 255)',
                showbackground = True,
                backgroundcolor = 'rgb(230, 230,230)'
            ),
            yaxis = dict(
                gridcolor = 'rgb(255, 255, 255)',
                zerolinecolor = 'rgb(255, 255, 255)',
                showbackground = True,
                backgroundcolor = 'rgb(230, 230,230)'
            ),
            zaxis = dict(
                gridcolor = 'rgb(255, 255, 255)',
                zerolinecolor = 'rgb(255, 255, 255)',
                showbackground = True,
                backgroundcolor = 'rgb(230, 230,230)'
            ),
            camera = dict(
                up = dict(
                    x = 0,
                    y = 0,
                    z = 1
                ),
                eye = dict(
                    x = -1.7428,
                    y = 1.0707,
                    z = 0.7100,
                )
            ),
            aspectratio = dict(x = 1, y = 1, z = 1),
            aspectmode = 'manual'
        )
    )
)

figure.show()

---